In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import tensorflow as ts
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, MaxPool1D, Flatten,Dropout,Input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Conv1D, MaxPooling1D
import numpy as np

Anormal veri setinin etiketlendirilmesi.

In [2]:
df_anomaly = pd.read_csv('data.csv')
df_anomaly.head()

,No.,Time,"""Source""",Destination,Protocol,Length,Info
0,1,"2025-01-12 20:57:22,463278",142.250.82.212,192.168.1.21,RTCP,110,Application specific subtype=13
1,2,"2025-01-12 20:57:22,463278",142.250.82.208,192.168.1.21,RTCP,110,Application specific subtype=13
2,3,"2025-01-12 20:57:22,463354",142.250.82.212,192.168.1.21,UDP,91,3478 > 58101 Len=49
3,4,"2025-01-12 20:57:22,470973",192.168.1.21,142.250.82.212,UDP,237,58101 > 3478 Len=195
4,5,"2025-01-12 20:57:22,477044",192.168.1.21,142.250.82.208,UDP,970,55959 > 3478 Len=928


In [3]:
df_anomaly.columns = df_anomaly.columns.str.strip().str.replace('"', '')

In [4]:
traffic_summary = df_anomaly.groupby(["Source", "Destination"]).size().reset_index(name="Packet Count")

In [5]:
attack_threshold = 1000
traffic_summary['Label'] = (traffic_summary['Packet Count'] > attack_threshold).astype(int)

In [6]:
def label_row(row):
    attack_protocols = ["UDP", "TCP"]  # Saldırılarda yaygın kullanılan protokoller
    if row["Protocol"] in attack_protocols and row["Length"] > 1000:
        return 1  # Saldırı
    else:
        return 0  # Normal

In [7]:
data = df_anomaly.merge(
    traffic_summary[['Source', 'Destination', 'Label']],
    on=['Source', 'Destination'],
    how='left'
)

In [8]:
data["Label"] = data.apply(label_row, axis=1)

In [9]:
output_path = 'anomaly.csv'
data.to_csv(output_path, index=False)

In [10]:
data.head(20)

,No.,Time,Source,Destination,Protocol,Length,Info,Label
0,1,"2025-01-12 20:57:22,463278",142.250.82.212,192.168.1.21,RTCP,110,Application specific subtype=13,0
1,2,"2025-01-12 20:57:22,463278",142.250.82.208,192.168.1.21,RTCP,110,Application specific subtype=13,0
2,3,"2025-01-12 20:57:22,463354",142.250.82.212,192.168.1.21,UDP,91,3478 > 58101 Len=49,0
3,4,"2025-01-12 20:57:22,470973",192.168.1.21,142.250.82.212,UDP,237,58101 > 3478 Len=195,0
4,5,"2025-01-12 20:57:22,477044",192.168.1.21,142.250.82.208,UDP,970,55959 > 3478 Len=928,0
5,6,"2025-01-12 20:57:22,477671",192.168.1.21,142.250.82.212,RTCP,90,Receiver Report,0
6,7,"2025-01-12 20:57:22,491641",192.168.1.21,142.250.82.212,UDP,233,58101 > 3478 Len=191,0
7,8,"2025-01-12 20:57:22,510177",192.168.1.21,142.250.82.212,RTCP,114,Receiver Report,0
8,9,"2025-01-12 20:57:22,510444",192.168.1.21,142.250.82.212,UDP,86,58101 > 3478 Len=44,0
9,10,"2025-01-12 20:57:22,511504",192.168.1.21,142.250.82.212,UDP,241,58101 > 3478 Len=199,0


Normal veri setinin etiketlendirilmesi.

In [11]:
df_normal = pd.read_csv('ağ.csv')
df_normal.head()

,No.,Time,Source,Destination,Protocol,Length,Info
0,1,"2025-01-06 10:31:34,261695",138.38.28.14,40.101.70.194,TLSv1.2,540,Application Data
1,2,"2025-01-06 10:31:34,261828",138.38.28.14,40.101.70.194,TLSv1.2,93,Application Data
2,3,"2025-01-06 10:31:34,266407",138.38.28.14,40.101.70.194,TCP,1514,55684 > 443 [ACK] Seq=526 Ack=1 Win=1020 Len...
3,4,"2025-01-06 10:31:34,266407",138.38.28.14,40.101.70.194,TCP,1514,55684 > 443 [ACK] Seq=1986 Ack=1 Win=1020 Le...
4,5,"2025-01-06 10:31:34,266407",138.38.28.14,40.101.70.194,TCP,1514,55684 > 443 [ACK] Seq=3446 Ack=1 Win=1020 Le...


In [12]:
df_normal.columns = df_normal.columns.str.strip().str.replace('"', '')

In [13]:
traffic_summary = df_normal.groupby(["Source", "Destination"]).size().reset_index(name="Packet Count")

In [14]:
attack_threshold = 1000
traffic_summary['Label'] = (traffic_summary['Packet Count'] > attack_threshold).astype(int)

In [15]:
def label_row(row):
    attack_protocols = ["UDP", "TCP"]  # Saldırılarda yaygın kullanılan protokoller
    if row["Protocol"] in attack_protocols and row["Length"] > 1000:
        return 1  # Saldırı
    else:
        return 0  # Normal

In [16]:
data_normal = df_normal.merge(
    traffic_summary[['Source', 'Destination', 'Label']],
    on=['Source', 'Destination'],
    how='left'
)

In [17]:
data_normal["Label"] = data_normal.apply(label_row, axis=1)

In [18]:
output_path = 'normal.csv'
data.to_csv(output_path, index=False)

In [19]:
data_normal.head(20)

,No.,Time,Source,Destination,Protocol,Length,Info,Label
0,1,"2025-01-06 10:31:34,261695",138.38.28.14,40.101.70.194,TLSv1.2,540,Application Data,0
1,2,"2025-01-06 10:31:34,261828",138.38.28.14,40.101.70.194,TLSv1.2,93,Application Data,0
2,3,"2025-01-06 10:31:34,266407",138.38.28.14,40.101.70.194,TCP,1514,55684 > 443 [ACK] Seq=526 Ack=1 Win=1020 Len...,1
3,4,"2025-01-06 10:31:34,266407",138.38.28.14,40.101.70.194,TCP,1514,55684 > 443 [ACK] Seq=1986 Ack=1 Win=1020 Le...,1
4,5,"2025-01-06 10:31:34,266407",138.38.28.14,40.101.70.194,TCP,1514,55684 > 443 [ACK] Seq=3446 Ack=1 Win=1020 Le...,1
5,6,"2025-01-06 10:31:34,266407",138.38.28.14,40.101.70.194,TLSv1.2,124,Application Data,0
6,7,"2025-01-06 10:31:34,274980",138.38.28.14,104.18.27.193,TCP,55,56376 > 443 [ACK] Seq=1 Ack=1 Win=250 Len=1,0
7,8,"2025-01-06 10:31:34,275069",138.38.28.14,34.36.216.150,TCP,55,56432 > 443 [ACK] Seq=1 Ack=1 Win=255 Len=1,0
8,9,"2025-01-06 10:31:34,322276",Apple_92:ec:ea,Broadcast,ARP,56,Who has 138.38.2.27? Tell 138.38.19.172,0
9,10,"2025-01-06 10:31:34,323644",40.101.70.194,138.38.28.14,TCP,60,443 > 55684 [ACK] Seq=1 Ack=526 Win=16385 Len=0,0


İki veri setinin birleştirilmesi.

In [20]:
merged_df = pd.concat([data, data_normal], axis=0)
merged_df.to_csv('dataset.csv', index=False)

In [21]:
X = merged_df.drop(columns=["Label"])  # Özellikler: Etiket sütunu hariç tüm sütunlar
y = merged_df["Label"]  # Etiketler: Saldırı (1) veya Normal (0)


In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [23]:
print(f"Eğitim seti boyutu: {len(X_train)}")
print(f"Test seti boyutu: {len(X_test)}")

Eğitim seti boyutu: 8734
Test seti boyutu: 2184


In [24]:
X_train.to_csv('X_train.csv', index=False)
X_test.to_csv('X_test.csv', index=False)
y_train.to_csv('y_train.csv', index=False)
y_test.to_csv('y_test.csv', index=False)

In [25]:
print(X_train.dtypes)
print(y_train.dtypes)

No.             int64
Time           object
Source         object
Destination    object
Protocol       object
Length          int64
Info           object
dtype: object
int64


In [26]:
import pandas as pd  
from keras.models import Sequential  
from keras.layers import Dense  

# 1. CSV dosyasını okuma  
X_test = pd.read_csv('X_test.csv')  

# 2. Time sütununu datetime formatına dönüştürme  
X_test['Time'] = pd.to_datetime(X_test['Time'], format='%Y-%m-%d %H:%M:%S,%f')  

# 3. Tarih özelliklerini çıkarma  
X_test['year'] = X_test['Time'].dt.year  
X_test['month'] = X_test['Time'].dt.month  
X_test['day'] = X_test['Time'].dt.day  
X_test['hour'] = X_test['Time'].dt.hour  
X_test['minute'] = X_test['Time'].dt.minute  
X_test['second'] = X_test['Time'].dt.second  
X_test['microsecond'] = X_test['Time'].dt.microsecond  

# 4. Sayısal olmayan sütunları çıkartma  
non_numeric_cols = ['No.', 'Time', 'Source', 'Destination', 'Protocol', 'Info']  
X_test = X_test.drop(columns=non_numeric_cols)  

# 5. Diğer boolean türündeki sütunları dönüştürme (eğer varsa)  
if 'some_boolean_column' in X_test.columns and X_test['some_boolean_column'].dtype == 'bool':  
    X_test['some_boolean_column'] = X_test['some_boolean_column'].astype(int)  

# 6. Sayısal sütunları float türüne dönüştürme  
numeric_cols = X_test.select_dtypes(include=['float64', 'int64']).columns  
X_test[numeric_cols] = X_test[numeric_cols].apply(pd.to_numeric, errors='coerce')  

In [27]:
import pandas as pd  
from keras.models import Sequential  
from keras.layers import Dense  

# 1. CSV dosyasını okuma  
X_train = pd.read_csv('X_train.csv')  

# 2. Time sütununu datetime formatına dönüştürme  
X_train['Time'] = pd.to_datetime(X_train['Time'], format='%Y-%m-%d %H:%M:%S,%f')  

# 3. Tarih özelliklerini çıkarma  
X_train['year'] = X_train['Time'].dt.year  
X_train['month'] = X_train['Time'].dt.month  
X_train['day'] = X_train['Time'].dt.day  
X_train['hour'] = X_train['Time'].dt.hour  
X_train['minute'] = X_train['Time'].dt.minute  
X_train['second'] = X_train['Time'].dt.second  
X_train['microsecond'] = X_train['Time'].dt.microsecond  

# 4. Sayısal olmayan sütunları çıkartma  
non_numeric_cols = ['No.', 'Time', 'Source', 'Destination', 'Protocol', 'Info']  
X_train = X_train.drop(columns=non_numeric_cols)  

# 5. Diğer boolean türündeki sütunları dönüştürme (eğer varsa)  
if 'some_boolean_column' in X_train.columns and X_train['some_boolean_column'].dtype == 'bool':  
    X_train['some_boolean_column'] = X_train['some_boolean_column'].astype(int)  

# 6. Sayısal sütunları float türüne dönüştürme  
numeric_cols = X_train.select_dtypes(include=['float64', 'int64']).columns  
X_train[numeric_cols] = X_train[numeric_cols].apply(pd.to_numeric, errors='coerce')  

In [28]:
print(X_train.dtypes)
print(X_test.dtypes)

Length         int64
year           int32
month          int32
day            int32
hour           int32
minute         int32
second         int32
microsecond    int32
dtype: object
Length         int64
year           int32
month          int32
day            int32
hour           int32
minute         int32
second         int32
microsecond    int32
dtype: object


In [33]:
model = Sequential() 
model.add(Conv1D(32, 9, padding="same",input_shape = (X_train.shape[1], 1), activation='relu'))
model.add(MaxPool1D(pool_size=(2)))
model.add(LSTM(units=16,return_sequences=False,dropout=0.2))
model.add(Dense(units=1))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=50, batch_size=250)

Epoch 1/50


c:\Users\esmabetul\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


35/35 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.4625 - loss: 4.1701
Epoch 2/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.4997 - loss: 3.1369
Epoch 3/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5763 - loss: 2.8361
Epoch 4/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5893 - loss: 3.1275
Epoch 5/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5931 - loss: 3.0324
Epoch 6/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5780 - loss: 2.7834
Epoch 7/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5598 - loss: 2.8670
Epoch 8/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5116 - loss: 2.4580
Epoch 9/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5461 - loss: 2.2318
Epoch 10/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5628 - loss: 2.0112
Epoch 11/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5412 - loss: 2.1829
Epoch 12/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4271 - loss: 2.5744


In [34]:
X_test = np.array(X_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
Y_test = np.array(y_test)

test_results = model.evaluate(X_test, Y_test, verbose=1)

69/69 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9554 - loss: 0.4500


In [37]:
model.save('lstm_model.h5')

In [35]:
import numpy as np  
import random  
import pandas as pd  

# Test veri setinden rastgele bir satır seç  
random_index = random.randint(0, X_test.shape[0] - 1)  
random_sample = X_test[random_index]  

# Model giriş boyutlarına uygun hale getirme  
random_sample_reshaped = np.reshape(random_sample, (1, random_sample.shape[0], 1))  

# Model tahmini yap  
prediction = model.predict(random_sample_reshaped, verbose=0)  

# Tahmini normal veya anormal olarak etiketleme  
if prediction[0][0] > 0.5:  
    print("Tahmin: Anormal (Attack)")  
else:  
    print("Tahmin: Normal (No Attack)")  

# Gerçek etiketi göster  
true_label = y_test.iloc[random_index]  # y_test bir pandas Series olduğu için iloc ile erişiyoruz.  

if true_label == 1:  
    true_label_text = 'Anormal (Attack)'  
else:  
    true_label_text = 'Normal (No Attack)'  

print(f"Gerçek Etiket: {true_label_text}")

Tahmin: Normal (No Attack)
Gerçek Etiket: Normal (No Attack)


In [43]:
from scapy.all import sniff  
import numpy as np  
from keras.models import load_model  
import sys  
from playsound import playsound  # Ses çalmak için playsound modülünü ekleyin  
from datetime import datetime  # Zaman bilgisi için datetime modülünü ekleyin  

# Önceden eğitilmiş modeli yükleyin  
model = load_model('lstm_model.h5')  # Model dosyanızın adını doğru belirtin  

# Protokol numarasını insan okuyabilir bir dizeye dönüştürme  
def get_protocol_name(proto):  
    if proto == 6:  
        return 'TCP'  
    elif proto == 17:  
        return 'UDP'  
    else:  
        return 'Diğer'  

# Saldırı tespit edildiğinde çıkış yapmak için bir bayrak  
attack_detected = False  

# Paketi yazdırmak için bir callback fonksiyonu  
def packet_callback(packet):  
    global attack_detected  # Global bayrak değişkenini kullan  

    try:  
        if packet.haslayer("IP"):  
            protocol_type = packet[0][1].proto  # Protokol numarasını al  
            src_ip = packet[0][1].src  
            dst_ip = packet[0][1].dst  
            src_bytes = len(packet)  # Paket boyutu  
            dst_bytes = 0  # Bu değeri ihtiyaçlarınıza göre ayarlayabilirsiniz  
            
            # Zaman bilgisini al  
            timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')  # Zamanı formatla  
        else:  
            src_ip = "No IP Layer"  
            dst_ip = "No IP Layer"  
            src_bytes = 0  
            dst_bytes = 0  
            protocol_type = 0  # Protokol bilgisi yok  
            timestamp = "No IP Layer"  # Zaman bilgisi yok  

        # Özellik vektörünü oluşturma  
        protocol_type_encoded = {  
            6: 1,   # TCP  
            17: 2,  # UDP  
        }.get(protocol_type, 0)  # Diğer tüm protokoller için varsayılan 0  

        feature_vector = np.array([src_bytes, dst_bytes, protocol_type_encoded]).reshape(1, -1)  

        # LSTM modelinin girişi için yeniden şekillendirme  
        feature_vector = np.reshape(feature_vector, (1, feature_vector.shape[1], 1))  

        # Model tahmini  
        prediction = model.predict(feature_vector, verbose=0)  
        
        # Protokol ismini al  
        protocol_name = get_protocol_name(protocol_type)  

        # Tahmini eşiğine göre sonuç  
        if prediction[0][0] == 1:  # Eşik değeri  
            print(f"[{timestamp}] Tahmin: Anormal (Attack) | Source: {src_ip}, Destination: {dst_ip}, Protocol: {protocol_name}, Length: {src_bytes}")  
            attack_detected = True  # Saldırı tespit edildi olarak ayarla  
            playsound('alarm.mp3')  # Alarm sesini çal (dosya yolunu doğru ayarlayın)  
            return  # Döngüyü kırmak için çık  

        else:  
            print(f"[{timestamp}] Tahmin: Normal (No Attack) | Source: {src_ip}, Destination: {dst_ip}, Protocol: {protocol_name}, Length: {src_bytes}")  

    except Exception as e:  
        print(f"Bu paketten özellikler çıkarılamadı. Hata: {e}")  

# Paket yakalamak için sniff fonksiyonunu kullanıyoruz  
sniff(iface='Wi-Fi', prn=packet_callback)  

# Eğer bir saldırı algılanırsa programı durdur  
if attack_detected:  
    print("Saldırı algılandı, alarm çalıp program sonlanıyor.")  
    sys.exit()  # Programdan çık

[2025-01-16 23:42:13] Tahmin: Normal (No Attack) | Source: 142.250.82.252, Destination: 192.168.1.21, Protocol: TCP, Length: 128
[2025-01-16 23:42:13] Tahmin: Normal (No Attack) | Source: 142.250.82.252, Destination: 192.168.1.21, Protocol: TCP, Length: 54
[2025-01-16 23:42:14] Tahmin: Normal (No Attack) | Source: 142.250.82.252, Destination: 192.168.1.21, Protocol: TCP, Length: 54
[2025-01-16 23:42:14] Tahmin: Normal (No Attack) | Source: 192.168.1.21, Destination: 142.250.82.252, Protocol: TCP, Length: 268
[2025-01-16 23:42:14] Tahmin: Normal (No Attack) | Source: 142.250.82.252, Destination: 192.168.1.21, Protocol: TCP, Length: 54
[2025-01-16 23:42:14] Tahmin: Normal (No Attack) | Source: 142.250.82.252, Destination: 192.168.1.21, Protocol: TCP, Length: 54
[2025-01-16 23:42:14] Tahmin: Normal (No Attack) | Source: 192.168.1.21, Destination: 142.250.82.252, Protocol: TCP, Length: 901
[2025-01-16 23:42:14] Tahmin: Normal (No Attack) | Source: 192.168.1.21, Destination: 142.250.82.252,

In [ ]:
from scapy.all import sniff
import numpy as np
from keras.models import load_model
import time

# Önceden eğitilmiş modeli yükleyin
model = load_model('lstm_model.h5')  # Model dosyanızın adı doğru olmalı

# DoS saldırısı için bir eşik değer tanımlayın
DOS_ATTACK_THRESHOLD = 0.3  # Modelin tahmin eşiği
ATTACK_COUNT_LIMIT = 10  # Belirli bir süre içinde kaç saldırının alarm oluşturacağını belirler
MONITOR_WINDOW = 10  # Saniye cinsinden izleme penceresi

# Saldırı sayacı ve zamanlayıcı
attack_counter = 0
start_time = time.time()

# Protokol numarasını insan okuyabilir bir dizeye dönüştürme
def get_protocol_name(proto):
    if proto == 6:
        return 'TCP'
    elif proto == 17:
        return 'UDP'
    else:
        return 'Diğer'

# Paketi analiz etmek için bir callback fonksiyonu
def packet_callback(packet):
    global attack_counter, start_time

    try:
        # Protokol ve kaynak/destination bilgilerini çıkarma
        if packet.haslayer("IP"):
            protocol_type = packet[0][1].proto  # Protokol numarasını al
            src_ip = packet[0][1].src
            dst_ip = packet[0][1].dst
            src_bytes = len(packet)  # Paket boyutu
            dst_bytes = 0  # Bu değeri ihtiyaçlarınıza göre ayarlayabilirsiniz
        else:
            src_ip = "No IP Layer"
            dst_ip = "No IP Layer"
            src_bytes = 0
            dst_bytes = 0
            protocol_type = 0  # Protokol bilgisi yok

        # Özellik vektörünü oluşturma
        protocol_type_encoded = {
            6: 1,   # TCP
            17: 2,  # UDP
        }.get(protocol_type, 0)  # Diğer tüm protokoller için varsayılan 0

        feature_vector = np.array([src_bytes, dst_bytes, protocol_type_encoded]).reshape(1, -1)

        # LSTM modelinin girişi için yeniden şekillendirme
        feature_vector = np.reshape(feature_vector, (1, feature_vector.shape[1], 1))

        # Model tahmini
        prediction = model.predict(feature_vector, verbose=0)

        # Protokol ismini al
        protocol_name = get_protocol_name(protocol_type)

        # Tahmini eşiğine göre sonuç
        if prediction[0][0] > DOS_ATTACK_THRESHOLD:  # Anormal paket tespiti
            print(f"ALARM: Anormal Trafik! | Source: {src_ip}, Destination: {dst_ip}, Protocol: {protocol_name}, Paket Boyutu: {src_bytes}")
            attack_counter += 1
        else:
            print(f"Normal Trafik | Source: {src_ip}, Destination: {dst_ip}, Protocol: {protocol_name}, Paket Boyutu: {src_bytes}")

        # İzleme penceresindeki saldırı sayısını kontrol et
        current_time = time.time()
        if current_time - start_time > MONITOR_WINDOW:  # İzleme penceresi süresi dolduğunda
            if attack_counter >= ATTACK_COUNT_LIMIT:
                print(f"Uyarı: {MONITOR_WINDOW} saniyede {attack_counter} saldırı tespit edildi! DoS saldırısı olabilir!")
            # Sayaçları sıfırla
            attack_counter = 0
            start_time = current_time

    except Exception as e:
        print(f"Bu paketten özellikler çıkarılamadı. Hata: {e}")

# Paket yakalamak için sniff fonksiyonunu kullanıyoruz
sniff(iface='Wi-Fi', prn=packet_callback, store=False)

Normal Trafik | Source: 192.168.1.21, Destination: 142.250.82.252, Protocol: TCP, Paket Boyutu: 262
Normal Trafik | Source: 142.250.82.252, Destination: 192.168.1.21, Protocol: TCP, Paket Boyutu: 128
Normal Trafik | Source: 142.250.82.252, Destination: 192.168.1.21, Protocol: TCP, Paket Boyutu: 101
Normal Trafik | Source: 192.168.1.21, Destination: 142.250.82.252, Protocol: TCP, Paket Boyutu: 279
Normal Trafik | Source: 192.168.1.21, Destination: 142.250.82.252, Protocol: TCP, Paket Boyutu: 1223
Normal Trafik | Source: 192.168.1.21, Destination: 142.250.82.252, Protocol: TCP, Paket Boyutu: 1223
Normal Trafik | Source: 192.168.1.21, Destination: 142.250.82.252, Protocol: TCP, Paket Boyutu: 1223
Normal Trafik | Source: 192.168.1.21, Destination: 142.250.82.252, Protocol: TCP, Paket Boyutu: 1224
Normal Trafik | Source: 192.168.1.21, Destination: 142.250.82.252, Protocol: TCP, Paket Boyutu: 1224
Normal Trafik | Source: 142.250.82.252, Destination: 192.168.1.21, Protocol: TCP, Paket Boyutu:

<Sniffed: TCP:0 UDP:0 ICMP:0 Other:0>